<a href="https://colab.research.google.com/github/AK18k/ex2_1/blob/main/ex2_latenight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import os
from google.colab import drive
#drive.mount('/content/drive')
DATA_PATH = '/content/ex2/data/ptb'
PATH = '/content/ex2'
os.chdir('/content/ex2')

FileNotFoundError: ignored

In [ ]:
!git clone https://github.com/AK18k/ex2_1

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


In [3]:
import os
import torch

from collections import Counter


class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.counter = Counter()
        self.total = 0

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        token_id = self.word2idx[word]
        self.counter[token_id] += 1
        self.total += 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r') as f:
            ids = torch.LongTensor(tokens)
            token = 0
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    ids[token] = self.dictionary.word2idx[word]
                    token += 1

        return ids


In [4]:
import math
import numpy as np
import matplotlib.pyplot as plt

class Result_Matrix:
    def __init__(self, model_type):
      self.model_name = model_type
      self.nepochs = 0
      self.train_ppl = np.array([])
      self.val_ppl = np.array([])
      self.test_ppl = np.array([])

    def add_result(self, result, result_type = 'train'):
      if result_type == 'train':
        self.train_ppl = np.append(self.train_ppl, result)
      elif result_type == 'val':
        self.val_ppl = np.append(self.val_ppl, result)
      elif result_type == 'test':
        self.test_ppl = np.append(self.test_ppl, result)

    def get_results(self, result_type = 'train'):
      if result_type == 'train':
        return(self.train_ppl)
      elif result_type == 'val':
        return(self.val_ppl)
      elif result_type == 'test':
        return(self.test_ppl) 


def plot_results(result_matrix):
  plt.figure()
  y = result_matrix.get_results('train')
  plt.plot(np.arange(1,len(y)+1), y, label = 'train', color = 'blue')
  y = result_matrix.get_results('val')
  plt.plot(np.arange(1,len(y)+1), y, label = 'val', color = 'green')
  y = result_matrix.get_results('test')
  plt.plot(np.arange(1,len(y)+1), y, label = 'test', color = 'red')
  plt.legend()
  plt.title(f'Model Type = {result_matrix.model_name}')
  #plt.ylim((0.75, 1))
  plt.xlabel('eoch')
  plt.ylabel('perplexity')
  plt.grid()
  plt.show()  


In [12]:


# First, let's define a custom Dataset to read the vectors of words
class PTBDataset(Dataset):
    def __init__(self, data, sequence_length=35):
        # In practice, `data` should be a list of integers representing the words in the PTB data set.
        self.data = [torch.tensor(item, dtype=torch.long) for item in data]
        self.sequence_length = sequence_length
   
    def __len__(self):
        return len(self.data) // self.sequence_length
   
    def __getitem__(self, idx):
        return self.data[idx * self.sequence_length: (idx + 1) * self.sequence_length]

# Collate function to pad sequences in the same batch to the same length
def collate(batch):
    return pad_sequence(batch, batch_first=True)

# Create a DataLoader
cor = Corpus('data/ptb/') .tokenize('data/ptb/') 
data = cor
dataset = PTBDataset(data)
dataloader = DataLoader(dataset, batch_size=20, collate_fn=collate, shuffle=True)

# Now let's define the LSTM language model
class LanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, nlayers = 2, dout = 0):
        super(LanguageModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, num_layers = nlayers, dropout = dout)
        self.linear = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embeddings(x)
        x, _ = self.lstm(x)
        x = self.linear(x)

        return x

# Create the model
vocab_size = 10000  # replace with your actual vocabulary size
embedding_dim = 300
hidden_dim = 512
model = LanguageModel(vocab_size, embedding_dim, hidden_dim)

# Define a loss function
loss_function = nn.CrossEntropyLoss()

# Define an optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
results_m = Result_Matrix('LSTM') 
for epoch in range(1):  # replace 10 with the number of epochs you want to train
    for inputs in dataloader:
        targets = inputs[:, 1:].contiguous()
        inputs = inputs[:, :-1].contiguous()

        # Forward pass
        outputs = model(inputs)
        outputs = outputs.view(-1, outputs.size(-1))

        targets = targets.view(-1)
       
        loss = loss_function(outputs, targets)
       
        # Calculate perplexity
        perplexity = torch.exp(loss)
        results_m.add_result(perplexity, 'train')
       
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
       
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}, Perplexity: {perplexity.item()}")

IsADirectoryError: ignored

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
